
# Exploration–Exploitation: Multi-Armed Bandits for Experimentation

This notebook complements classical A/B testing by introducing **multi-armed bandits**.

Instead of:

- Assigning a fixed share of traffic to each variant and analyzing at the end,

bandits aim to:

- **Exploit** arms that look good.  
- Still **explore** enough so you do not miss better options.

We will:

1. Define a simple **bandit environment** with Bernoulli rewards (e.g. click / no click).  
2. Implement several policies:
   - Greedy (always pick the current best).  
   - \(\varepsilon\)-greedy.  
   - UCB1 (Upper Confidence Bound).  
   - Thompson Sampling (Beta–Bernoulli).  
3. Compare them in terms of **cumulative reward** and **regret**.

The code is written to be reusable in other notebooks or small projects.


## 0) Setup

In [ ]:

from __future__ import annotations

from dataclasses import dataclass
from typing import List, Dict, Any, Callable

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (7, 4.5)
plt.rcParams["axes.grid"] = True



## 1) Bandit environment

We assume **K arms**, each with an unknown Bernoulli success probability:

\[
P(R=1 \mid A = k) = p_k.
\]

You can think of:

- Arms = different variants / creatives / prices.  
- Reward = click / conversion / success.

The environment holds the true \(p_k\) and generates rewards when a policy chooses arms.


In [ ]:

@dataclass
class BanditEnv:
    """Stateless K-armed bandit environment with Bernoulli rewards.

    Attributes
    ----------
    probs : np.ndarray
        True success probabilities for each arm (shape (K,)).
    rng : np.random.Generator
        Random number generator for reproducibility.
    """

    probs: np.ndarray
    rng: np.random.Generator

    def pull(self, arm: int) -> int:
        """Sample a reward (0/1) from the given arm.

        Parameters
        ----------
        arm : int
            Index of the arm to pull (0 <= arm < K).

        Returns
        -------
        int
            Reward (0 or 1).
        """
        if arm < 0 or arm >= self.probs.shape[0]:
            raise IndexError("arm index out of range")
        p = float(self.probs[arm])
        return int(self.rng.binomial(1, p))


def make_example_env(seed: int | None = 123) -> BanditEnv:
    """Create a simple 4-armed environment with fixed probabilities.

    For example:
    - Arm 0: 0.05
    - Arm 1: 0.06
    - Arm 2: 0.04
    - Arm 3: 0.08 (best)

    Parameters
    ----------
    seed : int | None
        Random seed for the environment.

    Returns
    -------
    BanditEnv
        Environment with fixed probabilities.
    """
    probs = np.array([0.05, 0.06, 0.04, 0.08], dtype=float)
    rng = np.random.default_rng(seed)
    return BanditEnv(probs=probs, rng=rng)


env = make_example_env()
env.probs



The **optimal arm** is the one with the highest success probability. In real life,
you do not know these probabilities and must learn them from data.



## 2) Bandit policies

We implement a tiny interface:

- Each policy keeps its own internal state (counts, estimates, prior parameters).  
- At each step:
  1. The policy chooses an arm.  
  2. The environment returns a reward.  
  3. The policy updates its state.

We will implement:

1. **Greedy**: always pick the arm with highest empirical mean.  
2. **\(\varepsilon\)-greedy**: explore with probability \(\varepsilon\), exploit otherwise.  
3. **UCB1**: pick arm with the largest upper confidence bound.  
4. **Thompson Sampling**: sample from Beta posteriors for each arm and pick the best sample.


In [ ]:

@dataclass
class BanditPolicyState:
    """State of a bandit policy at a given time.

    Attributes
    ----------
    counts : np.ndarray
        Number of times each arm was pulled.
    successes : np.ndarray
        Number of successes observed for each arm.
    """
    counts: np.ndarray
    successes: np.ndarray


class GreedyPolicy:
    """Greedy bandit policy: always pick the arm with highest empirical mean.

    If some arms have not been tried yet, it explores them first (round-robin).
    """

    def __init__(self, n_arms: int) -> None:
        self.n_arms = n_arms
        self.state = BanditPolicyState(
            counts=np.zeros(n_arms, dtype=int),
            successes=np.zeros(n_arms, dtype=int),
        )

    def select_arm(self) -> int:
        # Explore arms that have not been tried yet
        unexplored = np.where(self.state.counts == 0)[0]
        if unexplored.size > 0:
            return int(unexplored[0])

        # Exploit: choose arm with largest empirical mean
        means = self.state.successes / np.maximum(self.state.counts, 1)
        return int(np.argmax(means))

    def update(self, arm: int, reward: int) -> None:
        self.state.counts[arm] += 1
        self.state.successes[arm] += reward


In [ ]:

class EpsilonGreedyPolicy:
    """Epsilon-greedy policy.

    With probability epsilon, pick a random arm (exploration).
    With probability 1 - epsilon, pick the greedy arm based on empirical means.
    """

    def __init__(self, n_arms: int, epsilon: float = 0.1, seed: int | None = None) -> None:
        if not (0.0 <= epsilon <= 1.0):
            raise ValueError("epsilon must be in [0, 1].")
        self.n_arms = n_arms
        self.epsilon = float(epsilon)
        self.state = BanditPolicyState(
            counts=np.zeros(n_arms, dtype=int),
            successes=np.zeros(n_arms, dtype=int),
        )
        self.rng = np.random.default_rng(seed)

    def select_arm(self) -> int:
        # Explore
        if self.rng.uniform() < self.epsilon:
            return int(self.rng.integers(0, self.n_arms))

        # Otherwise behave like greedy
        unexplored = np.where(self.state.counts == 0)[0]
        if unexplored.size > 0:
            return int(unexplored[0])

        means = self.state.successes / np.maximum(self.state.counts, 1)
        return int(np.argmax(means))

    def update(self, arm: int, reward: int) -> None:
        self.state.counts[arm] += 1
        self.state.successes[arm] += reward


In [ ]:

class UCB1Policy:
    """UCB1 (Upper Confidence Bound) policy for Bernoulli bandits.

    At time t, choose arm a maximizing:

        mean_a + sqrt( 2 * log(t) / N_a )

    where mean_a is empirical mean and N_a is number of pulls of arm a.
    """

    def __init__(self, n_arms: int) -> None:
        self.n_arms = n_arms
        self.state = BanditPolicyState(
            counts=np.zeros(n_arms, dtype=int),
            successes=np.zeros(n_arms, dtype=int),
        )
        self.t = 0  # total number of pulls so far

    def select_arm(self) -> int:
        self.t += 1

        # Pull each arm once first
        unexplored = np.where(self.state.counts == 0)[0]
        if unexplored.size > 0:
            return int(unexplored[0])

        means = self.state.successes / np.maximum(self.state.counts, 1)
        bonus = np.sqrt(2.0 * math.log(self.t) / self.state.counts)
        ucb = means + bonus
        return int(np.argmax(ucb))

    def update(self, arm: int, reward: int) -> None:
        self.state.counts[arm] += 1
        self.state.successes[arm] += reward


In [ ]:

class ThompsonSamplingPolicy:
    """Thompson Sampling for Bernoulli bandits with Beta priors.

    Each arm a has a Beta(alpha_a, beta_a) posterior for its success probability.
    At each step:
    - sample theta_a ~ Beta(alpha_a, beta_a)
    - play arm with largest sampled theta_a
    - update alpha/beta based on reward
    """

    def __init__(self, n_arms: int, alpha0: float = 1.0, beta0: float = 1.0, seed: int | None = None) -> None:
        if alpha0 <= 0.0 or beta0 <= 0.0:
            raise ValueError("alpha0 and beta0 must be positive.")
        self.n_arms = n_arms
        self.alpha = np.full(n_arms, alpha0, dtype=float)
        self.beta = np.full(n_arms, beta0, dtype=float)
        self.rng = np.random.default_rng(seed)

    def select_arm(self) -> int:
        theta_samples = self.rng.beta(self.alpha, self.beta)
        return int(np.argmax(theta_samples))

    def update(self, arm: int, reward: int) -> None:
        if reward not in (0, 1):
            raise ValueError("reward must be 0 or 1 for Bernoulli TS.")
        if reward == 1:
            self.alpha[arm] += 1.0
        else:
            self.beta[arm] += 1.0



## 3) Simulation harness: one run

We now define a utility to simulate a **single bandit run**:

- Run for `T` steps.  
- At each step: policy chooses an arm, environment returns a reward.  
- Track:
  - reward per step,  
  - chosen arm,  
  - cumulative reward,  
  - cumulative regret.

Regret is defined with respect to the best arm:  

\[
\text{regret}_t = t \cdot p^* - \sum_{s=1}^t r_s,
\]

where \(p^* = \max_k p_k\).


In [ ]:

@dataclass
class BanditRunResult:
    rewards: np.ndarray
    arms: np.ndarray
    cum_reward: np.ndarray
    cum_regret: np.ndarray


def run_bandit(
    env: BanditEnv,
    policy,
    T: int,
) -> BanditRunResult:
    """Simulate a single bandit run.

    Parameters
    ----------
    env : BanditEnv
        Bandit environment with true probabilities.
    policy : object
        Policy object with methods `select_arm()` and `update(arm, reward)`.
    T : int
        Number of steps / pulls.

    Returns
    -------
    BanditRunResult
        Rewards, arms, and cumulative reward/regret over time.
    """
    K = env.probs.shape[0]
    best_prob = float(np.max(env.probs))

    rewards = np.zeros(T, dtype=float)
    arms = np.zeros(T, dtype=int)
    cum_reward = np.zeros(T, dtype=float)
    cum_regret = np.zeros(T, dtype=float)

    total = 0.0
    for t in range(T):
        arm = policy.select_arm()
        r = env.pull(arm)
        policy.update(arm, r)

        total += r
        rewards[t] = r
        arms[t] = arm
        cum_reward[t] = total
        cum_regret[t] = (t + 1) * best_prob - total

    return BanditRunResult(
        rewards=rewards,
        arms=arms,
        cum_reward=cum_reward,
        cum_regret=cum_regret,
    )



### Example: single run per policy


In [ ]:

env = make_example_env(seed=123)
T = 5000

policies = {
    "greedy": GreedyPolicy(env.probs.shape[0]),
    "eps_greedy_0.1": EpsilonGreedyPolicy(env.probs.shape[0], epsilon=0.1, seed=1),
    "ucb1": UCB1Policy(env.probs.shape[0]),
    "thompson": ThompsonSamplingPolicy(env.probs.shape[0], alpha0=1.0, beta0=1.0, seed=2),
}

results_single: Dict[str, BanditRunResult] = {}

for name, pol in policies.items():
    env_run = make_example_env(seed=123)  # fresh env for each policy
    res = run_bandit(env_run, pol, T=T)
    results_single[name] = res

# Plot cumulative regret for one run
plt.figure()
for name, res in results_single.items():
    plt.plot(res.cum_regret, label=name)
plt.xlabel("t")
plt.ylabel("cumulative regret")
plt.title("Single-run cumulative regret per policy")
plt.legend()
plt.tight_layout()
plt.show()



## 4) Multiple runs: average regret and arm allocation

Single runs can be noisy. To compare policies more robustly, we average over many runs:

- Repeat the simulation `n_runs` times per policy.  
- Compute the **mean cumulative regret** across runs.  
- Optionally analyze how often each arm was played.


In [ ]:

def run_many(
    env_probas: np.ndarray,
    policy_factory: Callable[[], Any],
    T: int,
    n_runs: int,
    seed: int | None = 999,
) -> Dict[str, Any]:
    """Run a bandit policy multiple times and collect statistics.

    Parameters
    ----------
    env_probas : np.ndarray
        True arm probabilities for the environment.
    policy_factory : Callable[[], Any]
        Function that returns a fresh policy instance.
    T : int
        Number of steps per run.
    n_runs : int
        Number of independent runs.
    seed : int | None
        Seed for the RNG used for environments.

    Returns
    -------
    dict
        Contains:
        - 'mean_cum_regret': np.ndarray
        - 'mean_cum_reward': np.ndarray
    """
    rng = np.random.default_rng(seed)
    K = env_probas.shape[0]
    best_prob = float(np.max(env_probas))

    cum_regrets = np.zeros((n_runs, T), dtype=float)
    cum_rewards = np.zeros((n_runs, T), dtype=float)

    for i in range(n_runs):
        # new env RNG seed for each run
        s = int(rng.integers(0, 1_000_000))
        env = BanditEnv(probs=env_probas.copy(), rng=np.random.default_rng(s))
        policy = policy_factory()

        res = run_bandit(env, policy, T=T)
        cum_regrets[i, :] = res.cum_regret
        cum_rewards[i, :] = res.cum_reward

    return {
        "mean_cum_regret": np.mean(cum_regrets, axis=0),
        "mean_cum_reward": np.mean(cum_rewards, axis=0),
    }


probs = make_example_env().probs
T = 5000
n_runs = 50

summary_multi: Dict[str, Dict[str, np.ndarray]] = {}

summary_multi["greedy"] = run_many(
    env_probas=probs,
    policy_factory=lambda: GreedyPolicy(probs.shape[0]),
    T=T,
    n_runs=n_runs,
)

summary_multi["eps_greedy_0.1"] = run_many(
    env_probas=probs,
    policy_factory=lambda: EpsilonGreedyPolicy(probs.shape[0], epsilon=0.1, seed=None),
    T=T,
    n_runs=n_runs,
)

summary_multi["ucb1"] = run_many(
    env_probas=probs,
    policy_factory=lambda: UCB1Policy(probs.shape[0]),
    T=T,
    n_runs=n_runs,
)

summary_multi["thompson"] = run_many(
    env_probas=probs,
    policy_factory=lambda: ThompsonSamplingPolicy(probs.shape[0], alpha0=1.0, beta0=1.0, seed=None),
    T=T,
    n_runs=n_runs,
)


In [ ]:

# Plot mean cumulative regret over runs
plt.figure()
for name, summ in summary_multi.items():
    plt.plot(summ["mean_cum_regret"], label=name)
plt.xlabel("t")
plt.ylabel("mean cumulative regret")
plt.title(f"Average cumulative regret over {n_runs} runs")
plt.legend()
plt.tight_layout()
plt.show()



In many settings you should see:

- **Greedy** can get stuck on suboptimal arms if early noise is unlucky.  
- **\(\varepsilon\)-greedy** keeps exploring and eventually finds the best arm, but exploration is constant.  
- **UCB1** and **Thompson Sampling** often achieve **lower regret**, especially as T grows.

This connects back to experimentation:
- A/B tests are like **fixed, non-adaptive** assignments.  
- Bandits are **adaptive**, assigning more traffic to better variants over time.

In practice you often mix both:

- Use bandits to **explore and optimize**.  
- Use hold-out A/B tests to **get clean causal estimates** and validate big launches.



## 5) How to reuse this in your projects

Ideas:

1. Replace the Bernoulli reward with:
   - click-through, sign-up, or purchase indicators from a simulator.  
   - synthetic revenue samples per arm.

2. Wrap policies in a small service interface:
   - keep `counts`, `successes`, or Beta parameters in a store.  
   - periodically update based on new events.

3. Combine with your **sequential / always-valid** tests:
   - use bandits for daily routing decisions,  
   - periodically run formal tests on logged data to make product decisions.

This notebook is intentionally minimal, so you can use it as a starting point
for a bandit playground or internal demos.
